In [1]:
# imports
import os, sys
import io
import time
import numpy as np
import xarray as xr
import dask
import dask.array as da
import rasterio
import arcpy

# import tools
#sys.path.append('../../../shared') temp for demo
sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\shared')
sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared')
import arc, tools, satfetcher

# import gdvspectra module
#sys.path.append('../../../modules') temp for demo
sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\modules')
sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
import cog

# globals 
AWS_KEY = ''
AWS_SECRET = ''
STAC_ENDPOINT = 'https://explorer.sandbox.dea.ga.gov.au/stac/search'
RESULT_LIMIT = 250

In [2]:
# get minimum bounding geom from input 
bbox = [119.32132692558059, -22.774853394924776, 119.34873641649256, -22.745010351081362]

# get collections based on platform 
collections = ['s2a_ard_granule', 's2b_ard_granule']

# prepare start and end date times
in_from_date = '2020-01-01'
in_to_date = '2020-12-31'

in_slc_off = False

RESULT_LIMIT = 250

# fetch stac data 
feats = cog.fetch_stac_data(stac_endpoint=STAC_ENDPOINT, 
                            collections=collections, 
                            start_dt=in_from_date, 
                            end_dt=in_to_date, 
                            bbox=bbox,
                            slc_off=in_slc_off,
                            limit=RESULT_LIMIT)

Beginning STAC search for items. This can take awhile.
Searching collection: s2a_ard_granule
Searching collection: s2b_ard_granule
Sorting result by time (old to new).
Found a total of 148 scenes.


In [26]:
in_bands = 'Blue;Green;Red;NIR1;SWIR2;SWIR3;OA_Mask'
in_platform = 'Sentinel'

# prepare band (i.e. stac assets) names
assets = arc.prepare_band_names(in_bands=in_bands, 
                                in_platform=in_platform)

In [27]:
in_epsg = 3577
in_res = 10
in_snap = True

# convert raw stac into dict with coord reproject, etc.
arcpy.SetProgressorLabel('Converting STAC data into useable format...')
meta, asset_table = cog.prepare_data(feats, 
                                     assets=assets,
                                     bounds_latlon=bbox, 
                                     bounds=None, 
                                     epsg=in_epsg, 
                                     resolution=in_res, 
                                     snap_bounds=in_snap,
                                     force_dea_http=True)

Converting raw STAC data into numpy format.
Converted raw STAC data successfully.


In [28]:
in_resampling = 'Nearest'
in_fill_value = '-999'
in_chunk_size = -1
in_dtype = 'int16'
in_rescale = True

# prepare resample and fill value types
resampling = in_resampling.lower()
fill_value = arc.prepare_fill_value_type(in_fill_value)

# convert assets to dask array
darray = cog.convert_to_dask(meta=meta, 
                             asset_table=asset_table, 
                             chunksize=in_chunk_size,
                             resampling=resampling, 
                             dtype=in_dtype, 
                             fill_value=fill_value, 
                             rescale=in_rescale)

Converting data into dask array.
Converted data successfully.


In [29]:
in_cell_align = 'Top-left'

# prepare alignment type
cell_align = arc.prepare_cell_align_type(in_cell_align)

# generate coordinates and dimensions from metadata
coords, dims = cog.build_coords(feats=feats,
                                assets=assets, 
                                meta=meta,
                                pix_loc=cell_align)

Creating dataset coordinates and dimensions.
Created coordinates and dimensions successfully.


In [30]:
# build final xarray data array
ds_name = 'stac-' + dask.base.tokenize(darray)
ds = xr.DataArray(darray,
                  coords=coords,
                  dims=dims,
                  name=ds_name
                  )

In [31]:
# prepare band (i.e. stac assets) names
assets = arc.prepare_band_names(in_bands=in_bands, 
                                in_platform=in_platform)

In [32]:
# comvert to cleaner xarray dataset
ds = ds.to_dataset(dim='band')

# append attributes onto dataset
ds = cog.build_attributes(ds=ds,
                          meta=meta, 
                          collections=collections, 
                          bands=assets,
                          slc_off=in_slc_off, 
                          bbox=bbox,
                          dtype=in_dtype,
                          snap_bounds=in_snap,
                          fill_value=fill_value, 
                          rescale=in_rescale,
                          cell_align=in_cell_align,
                          resampling=in_resampling)

Preparing and appending attributes to dataset.
Attributes appended to dataset successfully.


In [34]:
# get list of dataset vars and iterate compute on each
for counter, data_var in enumerate(list(ds.data_vars), start=1):

    # start clock
    start = time.time()

    # update progress bar
    print('Downloading band: {}...'.format(data_var))

    # compute!
    ds[data_var] = ds[data_var].compute()

    # notify time 
    duration = round(time.time() - start, 2)
    print('Band: {} took: {}s to download.'.format(data_var, duration)) 

Band: nbart_blue took: 53.4s to download.
Band: nbart_green took: 55.04s to download.
Band: nbart_red took: 63.84s to download.
Band: nbart_nir_1 took: 65.69s to download.
Band: nbart_swir_2 took: 16.69s to download.
Band: nbart_swir_3 took: 17.01s to download.
Band: fmask took: 10.91s to download.


In [36]:
ds

<xarray.Dataset>
Dimensions:       (time: 148, x: 311, y: 361)
Coordinates:
  * time          (time) datetime64[ns] 2020-01-02T02:23:55 ... 2020-12-29T02...
  * y             (y) float64 -2.514e+06 -2.514e+06 ... -2.517e+06 -2.517e+06
  * x             (x) float64 -1.288e+06 -1.288e+06 ... -1.285e+06 -1.285e+06
    spatial_ref   int32 3577
Data variables:
    nbart_blue    (time, y, x) int16 1277 1332 1475 1537 ... 508 520 508 512
    nbart_green   (time, y, x) int16 1407 1576 1629 1723 ... 724 723 730 728
    nbart_red     (time, y, x) int16 2031 2134 2207 2229 ... 1211 1244 1256 1273
    nbart_nir_1   (time, y, x) int16 2123 2254 2441 2550 ... 1709 1726 1767 1779
    nbart_swir_2  (time, y, x) int16 2978 3179 3179 3307 ... 2637 2637 2661 2661
    nbart_swir_3  (time, y, x) int16 2753 2954 2954 3130 ... 2145 2145 2145 2145
    fmask         (time, y, x) int16 2 2 2 2 2 2 2 2 2 2 ... 1 1 1 1 1 1 1 1 1 1
Attributes: (12/13)
    crs:               EPSG:3577
    grid_mapping:      spatial_ref
    transform:         (10.0, 0.0, -1288220.0, 0.0, -10.0, -2513880.0, 0.0, 0...
    nodatavals:        -999
    orig_collections:  ('s2a_ard_granule', 's2b_ard_granule')
    orig_bands:        ('nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir_...
    ...                ...
    orig_bbox:         (119.32132692558059, -22.774853394924776, 119.34873641...
    orig_dtype:        int16
    orig_snap_bounds:  True
    orig_cell_align:   Top-left
    orig_resample:     Nearest
    res:               10

<xarray.Dataset>
Dimensions:       (time: 45, x: 104, y: 121)
Coordinates:
  * time          (time) datetime64[ns] 2020-01-04T01:57:05 ... 2020-12-28T02...
  * y             (y) float64 -2.514e+06 -2.514e+06 ... -2.517e+06 -2.517e+06
  * x             (x) float64 -1.288e+06 -1.288e+06 ... -1.285e+06 -1.285e+06
    spatial_ref   int32 3577
Data variables:
    nbart_blue    (time, y, x) int16 ...
    nbart_green   (time, y, x) int16 ...
    nbart_red     (time, y, x) int16 ...
    nbart_nir     (time, y, x) int16 ...
    nbart_swir_1  (time, y, x) int16 ...
    nbart_swir_2  (time, y, x) int16 ...
    oa_fmask      (time, y, x) int16 ...
Attributes: (12/13)
    crs:               EPSG:3577
    grid_mapping:      spatial_ref
    transform:         [ 3.00000e+01  0.00000e+00 -1.28823e+06  0.00000e+00 -...
    nodatavals:        -999
    orig_collections:  ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
    orig_bands:        ['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir'...
    ...                ...
    orig_bbox:         [119.32132693 -22.77485339 119.34873642 -22.74501035]
    orig_dtype:        int16
    orig_snap_bounds:  True
    orig_cell_align:   Top-left
    orig_resample:     Nearest
    res:               30

In [1]:
# safe imports
import os, sys       # arcgis comes with these
import numpy as np   # arcgis comes with this
import pandas as pd  # arcgis comes with this

# risk imports (non-native to arcgis)
try:
    import xarray as xr  # not in arcgis
except Exception as e:
    arcpy.AddError('Python library Xarray is not installed.')
    raise ValueError(e)

# import tools
try:
    # shared scripts
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\shared')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared')
    import arc, satfetcher, tools

    # module scripts
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\modules')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
    import phenolopy, cog

except Exception as e:
    arcpy.AddError('Could not find tenement tools python scripts (modules, shared).')
    raise ValueError(e)

In [2]:
ds = xr.open_dataset(r"C:\Users\262272G\Desktop\test\yes.nc")

In [4]:
ds = tools.calculate_indices(ds=ds, 
                             index='MAVI'.lower(), 
                             custom_name='veg_idx', 
                             rescale=False, 
                             drop=True)

Calculating indices: mavi.
Calculating index: mavi
Renamed default indices.
Calculated indices successfully.


In [5]:
ds

<xarray.Dataset>
Dimensions:      (time: 56, x: 104, y: 121)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-05T02:03:11 ... 2020-11-10T02:...
  * y            (y) float64 -2.514e+06 -2.514e+06 ... -2.517e+06 -2.517e+06
  * x            (x) float64 -1.288e+06 -1.288e+06 ... -1.285e+06 -1.285e+06
    spatial_ref  int32 3577
Data variables:
    veg_idx      (time, y, x) float32 0.06968 0.05907 ... 0.06538 0.05773
Attributes: (12/13)
    crs:               EPSG:3577
    grid_mapping:      spatial_ref
    transform:         [ 3.00000e+01  0.00000e+00 -1.28823e+06  0.00000e+00 -...
    nodatavals:        nan
    orig_collections:  ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
    orig_bands:        ['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir'...
    ...                ...
    orig_bbox:         [119.32132693 -22.77485339 119.34873642 -22.74501035]
    orig_dtype:        int16
    orig_snap_bounds:  True
    orig_cell_align:   Top-left
    orig_resample:     Nearest
    res:               30

In [27]:
bu = ds.copy(deep=True)

In [45]:
index = 'mavi'
custom_name='veg_idx'
rescale = False
drop=True

In [29]:
# notify
print('Calculating indices: {0}.'.format(index))

# attempt da convert to ds, check for ds
was_da = False
if isinstance(ds, xr.DataArray):
    try:
        ds = ds.to_dataset(dim='variable')
        was_da = True
    except:
        raise TypeError('Failed to convert xarray DataArray to Dataset. Please provide a Dataset.')

elif not isinstance(ds, xr.Dataset):
    raise TypeError('Not an xarray dataset. Please provide Dataset.')

Calculating indices: mavi.


In [30]:
# prepare index, custom names if empty
index = index if index is not None else []
custom_name = custom_name if custom_name is not None else []

# if not none but not list, prepare also
indices = index if isinstance(index, list) else [index]
custom_names = custom_name if isinstance(custom_name, list) else [custom_name]

# check if custon names same length as index, if provided
if custom_names and len(indices) != len(custom_names):
    raise ValueError('Custom names must be provided for all indexes.')

In [34]:
# get pre-processing band names for drop later
drop_bands = list(ds.data_vars)

# create copy ds
ds = ds.copy(deep=True)

In [37]:
# calc index/indices
for index in indices:
    try:
        # notify
        print('Calculating index: {0}'.format(index))

        # ndvi
        if index == 'ndvi':
            ds['ndvi'] = ((ds.nir - ds.red) / 
                          (ds.nir + ds.red))

        # evi with normalise
        elif index == 'evi':
            f = lambda ds: ((2.5 * (ds.nir - ds.red)) / 
                            (ds.nir + 6 * ds.red - 7.5 * ds.blue + 1))
            ds['evi'] = f(ds / 10000.0)

        # savi with normalise
        elif index == 'savi':       
            f = lambda ds: ((1.5 * (ds.nir - ds.red)) /
                            (ds.nir + ds.red + 0.5))
            ds['savi'] = f(ds / 10000.0)

        # msavi with normalise
        elif index == 'msavi':       
            f = lambda ds: ((2 * ds.nir + 1 - ((2 * ds.nir + 1)**2 - 
                             8 * (ds.nir - ds.red))**0.5) / 2)
            ds['msavi'] = f(ds / 10000.0)

        # slavi
        elif index == 'slavi':
            ds['slavi'] = ((ds.nir) / 
                           (ds.red + ds.swir2))

        # mavi
        elif index == 'mavi':
            ds['mavi'] = ((ds.nir - ds.red) / 
                          (ds.nir + ds.red + ds.swir1))

        # kndvi (non-linear ndvi)
        elif index == 'kndvi':
            ds['kndvi'] = np.tanh(((ds.nir - ds.red) /
                                   (ds.nir + ds.red)) ** 2)

        # tass cap greenness (Crist 1985 coeffs) with normalise
        elif index == 'tcg':        
            f = lambda ds: ((-0.1603 * ds.blue + -0.2819 * ds.green +
                             -0.4934 * ds.red + 0.7940 * ds.nir +
                             -0.0002 * ds.swir1 + -0.1446 * ds.swir2))
            ds['tcg'] = f(ds / 10000.0)

        # tass cap brightness (Crist 1985 coeffs) with normalise
        elif index == 'tcb':
            f = lambda ds: ((0.2043 * ds.blue + 0.4158 * ds.green +
                             0.5524 * ds.red + 0.5741 * ds.nir +
                             0.3124 * ds.swir1 + -0.2303 * ds.swir2))
            ds['tcb'] = f(ds / 10000.0)

        # tass cap wetness (Crist 1985 coeffs) with normalise
        elif index == 'tcw':       
            f = lambda ds: ((0.0315 * ds.blue + 0.2021 * ds.green +
                             0.3102 * ds.red + 0.1594 * ds.nir +
                             -0.6806 * ds.swir1 + -0.6109 * ds.swir2))
            ds['tcw'] = f(ds / 10000.0)

        # ndmi
        elif index == 'ndmi':
            ds['ndmi'] = ((ds.nir - ds.swir1) / 
                          (ds.nir + ds.swir1))

        # gvmi
        elif index == 'gvmi':
            ds['gvmi'] = (((ds.nir + 0.1) - (ds.swir2 + 0.02)) / 
                          ((ds.nir + 0.1) + (ds.swir2 + 0.02)))

        # nbr
        elif index == 'nbr':
            ds['nbr'] = ((ds.nir - ds.swir2) /
                        (ds.nir + ds.swir2))

    except:
        raise ValueError('Could not calculate: {0}. Please check bands.'.format(index))


Calculating index: mavi


In [49]:
# rescale -1 to 1 -> 0 to 2 if requested
if rescale:
    for index in indices:
        if index not in ['slavi', 'kndvi', 'tcg', 'tcb', 'tcw', 'nbr']:
            ds[index] = ds[index] + 1
    print('Rescaled index values from -1 to 1 -> 0 to 2.')

# rename bands if requested
if custom_names:
    try:
        ds = ds.rename(dict(zip(indices, custom_names)))
        print('Renamed default indices.')
    except:
        print('Warning: could not rename bands. Please check.')

# drop original bands if requested
if drop:
    ds = ds.drop(drop_bands, errors='ignore')
    
# convert back to datarray
if was_da:
    ds = ds.to_array()
    
# notify and return
print('Calculated indices successfully.')
#return ds

Calculated indices successfully.
